### Importing all dependencies

In [ ]:
from pydaalcontrib.nn import *
from pydaalcontrib.model.nn import Model, Relu, SigmoidCrossEntropy
from daal.algorithms.optimization_solver import adagrad
import numpy as np

### Definition of a DNN model

In [ ]:
x = conv2d([3,3,6], strides=[2,2], paddings=[2,2])
x = x(max_pool2d([2,2]))(Relu())(dropout(.5))
x = x(conv2d([3,3,12], strides=[2,2], paddings=[2,2]))
x = x(max_pool2d([2,2]))(Relu())(dropout(.5))
x = x(conv2d([3,3,24], strides=[2,2], paddings=[2,2]))
x = x(avg_pool2d([2,2]))(Relu())
x = x(fc(10))(lrn())(fc(1))
x = x(SigmoidCrossEntropy())

model = Model(x)

### Create superficial data and labels

In [ ]:
num_epochs = 10
np.random.seed(0)
labels = np.append(np.zeros(100), np.ones(100))
data = np.random.rand(200, 3, 60, 60)
data[:100] = data[:100]*-1

### Build a net with the Adagrad solver

In [ ]:
net = DAALNet().build(model).with_solver(adagrad.Batch())

### Train only for 10 epochs

In [ ]:
for i in range(num_epochs):
    perm = np.random.permutation(200)
    net.train(data[perm], labels[perm], batch_size=10)

### Run forward pass again to obtain predictions (class probabilities)

In [ ]:
with net.predict(data) as predictions:
    predicted = np.all(predictions > 0.5, axis=1)
    print('Probabilities  : %s' % np.round(predictions[perm[:10]].ravel(), decimals=3))
    print('Predicted class: %s' % predicted[perm[:10]].astype(int))
    print('True class     : %s' % labels[perm[:10]].astype(int))